In [ ]:
############################  example of loading PRISM data ###################################
import sys 
sys.path.append('/data/SWATGenXApp/codes/HydroGeoDataset')
from core import DataImporter
import matplotlib.pyplot as plt
import numpy as np

## NOTE: method for downscaling PRISM is nearest neighbor and have done with Arcpy. Note thate resampling of PRISM happened beforer H5 creation
## PRISM data is in mm/day, C, C

config = { 
    "RESOLUTION": 250,  ## resolution of the PRISM data
    "huc8": None,
    "video": False, ## if True, it will show the video of the PRISM data
    "aggregation": "annual", ## aggregation of the PRISM data can be daily, monthly, yearly
    "start_year": 2000,
    "end_year": 2001,
    'bounding_box': [-85.444332, 43.658148, -85.239256, 44.164683], # min_longitude, min_latitude, max_longitude, max_latitude
}
importer = DataImporter(config)
############################## example of loading PRISM data ###################################
pr_prism, tmax_prism, tmin_prism = importer.PRISM() 

In [ ]:
####################### example of loading LOCA2 data ###################################
import sys 
sys.path.append('/data/SWATGenXApp/codes/HydroGeoDataset')
from core import DataImporter
import os 
import numpy as np

## NOTE: method for downscaling LOCA2 is nearest neighbor and have done with numpy opeations. Note that the resampling of LOCA2 happens in HydroGeoDataset.py
## NOTE: actual data unit in LOCA2 is kg m-2 s-1 for precipitation and K for temperature. We convert them to mm and C respectively in HydroGeoDataset.py

config = {
        "RESOLUTION": 250,
        "huc8": None,
        "video": False,
        "aggregation": "monthly",
        'bounding_box': [-85.444332, 43.658148, -85.239256, 44.164683], # min_longitude, min_latitude, max_longitude, max_latitude
    }

importer = DataImporter(config)
### NOTE: the list of all models and their ensemble is in /data/LOCA2/list_of_all_models.txt
start_year = 2000
end_year = 2001
cc_model = "ACCESS-CM2"
scenario = "historical"
ensemble = "r2i1p1f1"

ppt_loca2, tmax_loca2, tmin_loca2 = importer.LOCA2(start_year=start_year, end_year=end_year, cc_model= cc_model, scenario=scenario, ensemble=ensemble, cc_time_step='daily')


In [ ]:
import sys 
from core import DataImporter

### get MODIS Evapotranspiration data
config = {
    "RESOLUTION": 250,
    "huc8": None,
    "video": False,
}

importer = DataImporter(config)
start_year = 2000
end_year = 2002
et_modis = importer.MODIS_ET(start_year=start_year, end_year=end_year,  h5_group_name="MODIS/MODIS_ET")

In [ ]:
	
import sys 
sys.path.append('/data/SWATGenXApp/codes/HydroGeoDataset')
from HydroGeoDataset.core import DataImporter

config = {
		"RESOLUTION": 250,
		"huc8": "4060105",
		"video": True,
	}
importer = DataImporter(config)

gw_3d_ds = importer.gw_3d_ds(start_year=2020, end_year=2021)

In [ ]:

import sys 
sys.path.append('/data/SWATGenXApp/codes/HydroGeoDataset')
from HydroGeoDataset.core import DataImporter

config = {
		"RESOLUTION": 250,
		
		"geospatial": True,
	}

importer = DataImporter(config)

gw_station_data = importer.gw_stations_ds(start_year=1990, end_year=2021)

print(f"numerical feature: {gw_station_data['421332085401901_1609_389']['numerical_feature']}")
print(f"categorical feature: {gw_station_data['421332085401901_1609_389']['categorical_feature']}")
print(f"head: {gw_station_data['421332085401901_1609_389']}")



In [ ]:
import sys 
sys.path.append('/data/SWATGenXApp/codes/HydroGeoDataset')
from HydroGeoDataset.core import DataImporter


config = {
	"RESOLUTION": 250,
	"PFAS": "PFOS"}   ### othr PFAS: PFHxS, PFOA, PFNA, PFDA, PFOS
importer = DataImporter(config)

pfas_max, pfas_mean, pfas_std = importer.import_pfas_data()

print(f"pfas_max: {pfas_max}")
print(f"pfas_mean: {pfas_mean}")
print(f"pfas_std: {pfas_std}")

In [ ]:
	
import sys 
sys.path.append('/data/SWATGenXApp/codes/HydroGeoDataset')
from HydroGeoDataset.core import DataImporter

extract_point_data = False
if extract_point_data:
	config = {
		"RESOLUTION": 250}

	input_path = "/data2/MyDataBase/HuronRiverPFAS/Huron_Biosolid_sites.geojson"
	output_path = "/data/MyDataBase/test.pkl"
	importer = DataImporter(config)
	gdf = importer.extract_features(input_path)
	gdf.to_pickle(output_path)
	gdf.to_file(output_path.replace(".pkl", ".shp"))

single_location_extraction = True
if single_location_extraction:
	### a random location within Michigan LP
	config = {
		"RESOLUTION": 250}
	importer = DataImporter(config)
	lat, lon = 42.0, -84.0
	gdf = importer.extract_features(single_location=(lat, lon))

	print(f"Extracted features: {gdf.columns}")

# Plot the shapefile
#gdf.plot()
#plt.savefig("input_figs/P_locations_rasters_30m.png")

In [ ]:
# import snowdas data

import sys 
sys.path.append('/data/SWATGenXApp/codes/HydroGeoDataset')
from HydroGeoDataset.core import DataImporter
config = {
	"RESOLUTION": 250
    }  

importer = DataImporter(config)

data = importer.extract_snowdas_data(snowdas_var='snow_layer_thickness', year = 2015)  #'melt_rate', 'snow_accumulation', 'snow_layer_thickness', 'snow_water_equivalent', 'snowpack_sublimation_rate'. data range from 2004 to 2019

print(f"Data shape: {data.shape}")  


In [ ]:
### Extracting the data for a specific location without using the DataImporter class
from HydroGeoDataset.core import get_rowcol_index_by_latlon, get_rowcol_range_by_latlon, read_h5_file, hydrogeo_dataset_dict
import h5py 
import numpy as np
import time


if __name__ == '__main__':

    path = "/data/MyDataBase/HydroGeoDataset_ML_250.h5"

    dic = hydrogeo_dataset_dict()

    print(dic['geospatial'])    
    time.sleep(10)  
    hydrogeo_dic = hydrogeo_dataset_dict(path)

    lat, lon = 42.0, -84.0
    row, col = get_rowcol_index_by_latlon(lat, lon, 250)
    for key in hydrogeo_dic['geospatial']:
        if key in ["x_250m", "y_250m", "lat_250m", "lon_250m", "mask_250m"]:
            continue
        address = f"geospatial/{key}"

        data = read_h5_file(lat=row, lon=col, address=address)
        print(f"key: {key}, data: {data:.2f}")


        min_lat, max_lat = 41.5, 42.5
        min_lon, max_lon = -84.5, -83.5
        min_row_number, max_row_number, min_col_number, max_col_number = get_rowcol_range_by_latlon(min_lat, max_lat, min_lon, max_lon)
        data = read_h5_file(address, lat_range=[min_row_number, max_row_number], lon_range=[min_col_number, max_col_number])
        print(f"key: {key}, average data: {data.mean():.2f}, median data: {np.median(data):.2f}, std data: {data.std():.2f}")
        